In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error
import re

In [48]:
df = pd.read_csv('akabane.csv', sep=',', skiprows=0, header=0)
print(df.shape)
print(df.info())
display(df.head())

(298, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   chinRyo     298 non-null    int64  
 1   ekiToho     298 non-null    int64  
 2   senMenseki  298 non-null    float64
 3   chiku       298 non-null    int64  
 4   kai_all     298 non-null    int64  
 5   shinchiku   298 non-null    int64  
 6   kai_above   298 non-null    int64  
 7   south       298 non-null    int64  
 8   parking     298 non-null    int64  
 9   autolock    298 non-null    int64  
 10  aircon      298 non-null    int64  
 11  bathtoilet  298 non-null    int64  
 12  addbatch    298 non-null    int64  
 13  flooring    298 non-null    int64  
 14  pet         298 non-null    int64  
 15  url_detail  298 non-null    object 
dtypes: float64(1), int64(14), object(1)
memory usage: 37.4+ KB
None


,chinRyo,ekiToho,senMenseki,chiku,kai_all,shinchiku,kai_above,south,parking,autolock,aircon,bathtoilet,addbatch,flooring,pet,url_detail
0,55000,19,19.80,40,2,0,1,1,0,0,1,1,1,1,1,https://www.chintai.net/detail/bk-000000663000...
1,72000,1,42.00,23,3,0,1,0,0,0,1,1,0,1,0,https://www.chintai.net/detail/bk-000000428000...
2,48000,10,14.16,34,5,0,1,0,0,0,1,0,0,1,0,https://www.chintai.net/detail/bk-C01007529004...
3,67000,11,19.22,12,4,0,1,0,0,0,1,1,0,1,0,https://www.chintai.net/detail/bk-000000002000...
4,114000,4,45.36,2,4,0,0,0,0,1,1,1,1,1,0,https://www.chintai.net/detail/bk-000000002000...


In [49]:
df=df.drop(columns='url_detail') 
X = df.drop(columns='chinRyo')  # explanatory variables
y = df['chinRyo']  # objective variable, 1D
print('X:', X.shape)
display(X.head())
print('y:', y.shape)
print(y.head())

X: (298, 14)


,ekiToho,senMenseki,chiku,kai_all,shinchiku,kai_above,south,parking,autolock,aircon,bathtoilet,addbatch,flooring,pet
0,19,19.80,40,2,0,1,1,0,0,1,1,1,1,1
1,1,42.00,23,3,0,1,0,0,0,1,1,0,1,0
2,10,14.16,34,5,0,1,0,0,0,1,0,0,1,0
3,11,19.22,12,4,0,1,0,0,0,1,1,0,1,0
4,4,45.36,2,4,0,0,0,0,1,1,1,1,1,0


y: (298,)
0     55000
1     72000
2     48000
3     67000
4    114000
Name: chinRyo, dtype: int64


In [50]:
X_c = sm.add_constant(X)
model = sm.OLS(y, X_c)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                chinRyo   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     109.7
Date:                Mon, 06 Oct 2025   Prob (F-statistic):          2.14e-105
Time:                        17:25:08   Log-Likelihood:                -3175.3
No. Observations:                 298   AIC:                             6381.
Df Residuals:                     283   BIC:                             6436.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.49e+04   4398.960      3.388      0.0

In [51]:
print(results.rsquared)
print(results.rsquared_adj)
print(results.params)

0.8444310467731203
0.8367350561541227
const         14902.362651
ekiToho        -250.885115
senMenseki     1478.705651
chiku           -86.793464
kai_all        1021.932079
shinchiku      9400.969269
kai_above      3683.373118
south          1348.609666
parking      -15515.080028
autolock       4921.565990
aircon         9127.092542
bathtoilet     2753.491259
addbatch       3855.051776
flooring        454.077421
pet            -996.915151
dtype: float64


In [52]:
X_scaled_ar = scale(X)
y_scaled_ar = scale(y)
X_scaled = pd.DataFrame(X_scaled_ar, columns=X.columns)
y_scaled = pd.Series(y_scaled_ar, name=y.name)
model = sm.OLS(y_scaled, X_scaled)
results_scaled = model.fit()
print(results_scaled.summary())

                                 OLS Regression Results                                
Dep. Variable:                chinRyo   R-squared (uncentered):                   0.844
Model:                            OLS   Adj. R-squared (uncentered):              0.837
Method:                 Least Squares   F-statistic:                              110.1
Date:                Mon, 06 Oct 2025   Prob (F-statistic):                   8.62e-106
Time:                        17:25:08   Log-Likelihood:                         -145.60
No. Observations:                 298   AIC:                                      319.2
Df Residuals:                     284   BIC:                                      371.0
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [53]:
print(results_scaled.params.sort_values(key=np.abs, ascending=False))

senMenseki    0.874595
parking      -0.184591
kai_all       0.098115
aircon        0.095445
autolock      0.089945
shinchiku     0.073792
addbatch      0.061516
kai_above     0.061428
bathtoilet    0.047777
chiku        -0.043009
ekiToho      -0.038585
south         0.024912
pet          -0.009360
flooring      0.006072
dtype: float64


In [54]:
X_test = pd.DataFrame([ [8,45.00, 10,3,0,3,1,0,1,1,1,1,1,0]
                      ],
                      columns=X.columns)  # example
print('X for prediction:')
display(X_test)

X for prediction:


,ekiToho,senMenseki,chiku,kai_all,shinchiku,kai_above,south,parking,autolock,aircon,bathtoilet,addbatch,flooring,pet
0,8,45.0,10,3,0,3,1,0,1,1,1,1,1,0


In [55]:
X_test_c = sm.add_constant(X_test, has_constant='add')
y_test = results.predict(X_test_c)
print('Predicted y: {0:.2f}'.format(y_test[0]))

Predicted y: 115144.91


In [56]:
y_pred = results.predict(X_c)
print(y_pred.head())

0     58210.823587
1     93844.695772
2     48755.209224
3     59627.590079
4    105998.330874
dtype: float64


In [57]:
mse = mean_squared_error(y, y_pred)
print('MSE, RMSE:', mse, np.sqrt(mse))

MSE, RMSE: 105344304.17489637 10263.737339531657


In [58]:
d_scaled=pd.concat([X_scaled,y_scaled],axis=1)
display(d_scaled.head())

,ekiToho,senMenseki,chiku,kai_all,shinchiku,kai_above,south,parking,autolock,aircon,bathtoilet,addbatch,flooring,pet,chinRyo
0,2.769550,-0.600997,1.535134,-0.856933,-0.213574,0.579934,1.326371,-0.346844,-0.726777,0.295958,0.631713,1.874874,0.405046,3.831998,-0.714065
1,-1.728139,0.841396,0.216777,-0.456673,-0.213574,0.579934,-0.753937,-0.346844,-0.726777,0.295958,0.631713,-0.533369,0.405046,-0.260960,-0.060777
2,0.520706,-0.967443,1.069831,0.343848,-0.213574,0.579934,-0.753937,-0.346844,-0.726777,0.295958,-1.582998,-0.533369,0.405046,-0.260960,-0.983066
3,0.770577,-0.638681,-0.636278,-0.056412,-0.213574,0.579934,-0.753937,-0.346844,-0.726777,0.295958,0.631713,-0.533369,0.405046,-0.260960,-0.252920
4,-0.978524,1.059704,-1.411782,-0.056412,-0.213574,-1.724336,-0.753937,-0.346844,1.375937,0.295958,0.631713,1.874874,0.405046,-0.260960,1.553230


In [59]:
def step_aic_forward(model, exog, endog, **kwargs):

    exog = np.r_[[exog]].flatten()
    endog = np.r_[[endog]].flatten()
    remaining = set(exog)
    selected = []  

    formula_head = 'Q("' + '") + Q("'.join(endog) + '") ~ '
    formula = formula_head + '1'
    aic = model(formula=formula, **kwargs).fit().aic
    print('AIC: {:.3f}, formula: {}'.format(aic, formula))

    current_score, best_new_score = aic, aic

    while True:
        score_with_candidates = []
        for candidate in remaining:
            formula_tail = 'Q("' + '") + Q("'.join(selected + [candidate]) + '")'
            formula = formula_head + formula_tail
            aic = model(formula=formula, **kwargs).fit().aic
            print('AIC: {:.3f}, formula: {}'.format(aic, formula))

            score_with_candidates.append((aic, candidate))

        score_with_candidates.sort()
        best_score, best_candidate = score_with_candidates[0]

        improved = False
        if best_score < current_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_score
            improved = True
            
        if not remaining or not improved: break

    formula = formula_head + 'Q("' + '") + Q("'.join(selected) + '")'
    print('The best formula: {}'.format(formula))
    aic = model(formula=formula, **kwargs).fit().aic
    print('Minimum AIC: {:.3f}'.format(aic))
    
    ret = model(formula, **kwargs).fit()
    ret.model.exog_names_org = [re.sub(r'Q\(\"(.*)\"\)',r'\1',x) for x in list(ret.model.exog_names)]
    ret.model.endog_names_org = re.sub(r'Q\(\"(.*)\"\)',r'\1',ret.model.endog_names)
    return ret

In [60]:
header_y = y_scaled.name
header_x = X_scaled.columns
model = step_aic_forward(smf.ols, header_x,header_y, data=d_scaled)

AIC: 847.687, formula: Q("chinRyo") ~ 1
AIC: 845.415, formula: Q("chinRyo") ~ Q("flooring")
AIC: 841.110, formula: Q("chinRyo") ~ Q("parking")
AIC: 441.885, formula: Q("chinRyo") ~ Q("senMenseki")
AIC: 806.658, formula: Q("chinRyo") ~ Q("kai_all")
AIC: 846.147, formula: Q("chinRyo") ~ Q("chiku")
AIC: 844.871, formula: Q("chinRyo") ~ Q("south")
AIC: 836.173, formula: Q("chinRyo") ~ Q("kai_above")
AIC: 846.555, formula: Q("chinRyo") ~ Q("ekiToho")
AIC: 846.361, formula: Q("chinRyo") ~ Q("aircon")
AIC: 778.840, formula: Q("chinRyo") ~ Q("bathtoilet")
AIC: 849.015, formula: Q("chinRyo") ~ Q("pet")
AIC: 734.128, formula: Q("chinRyo") ~ Q("addbatch")
AIC: 842.364, formula: Q("chinRyo") ~ Q("autolock")
AIC: 849.580, formula: Q("chinRyo") ~ Q("shinchiku")
AIC: 438.592, formula: Q("chinRyo") ~ Q("senMenseki") + Q("flooring")
AIC: 407.516, formula: Q("chinRyo") ~ Q("senMenseki") + Q("parking")
AIC: 410.983, formula: Q("chinRyo") ~ Q("senMenseki") + Q("kai_all")
AIC: 419.365, formula: Q("chinRyo"

In [61]:
X_scaled2 = X_scaled[['senMenseki', 'autolock', 'parking', 'aircon', 'shinchiku', 'kai_all', 'kai_above', 'bathtoilet', 'addbatch']]

mod2 = sm.OLS(y_scaled,X_scaled2)
res2 = mod2.fit()
print(res2.summary())

                                 OLS Regression Results                                
Dep. Variable:                chinRyo   R-squared (uncentered):                   0.842
Model:                            OLS   Adj. R-squared (uncentered):              0.837
Method:                 Least Squares   F-statistic:                              171.4
Date:                Mon, 06 Oct 2025   Prob (F-statistic):                   2.39e-110
Time:                        17:25:10   Log-Likelihood:                         -147.71
No. Observations:                 298   AIC:                                      313.4
Df Residuals:                     289   BIC:                                      346.7
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [62]:
num_cols = mod2.exog.shape[1] 
vifs = [variance_inflation_factor(mod2.exog, i)
        for i in range(0, num_cols)]
pd.DataFrame(vifs, index=mod2.exog_names, columns=["VIF"])

,VIF
senMenseki,2.259230
autolock,1.518966
parking,1.225613
aircon,1.102048
shinchiku,1.039492
kai_all,1.637924
kai_above,1.112906
bathtoilet,1.365743
addbatch,1.684562


In [63]:
X_final = X[['senMenseki', 'autolock', 'parking', 'aircon', 'shinchiku', 'kai_all', 'kai_above', 'bathtoilet', 'addbatch']]

mod_final = sm.OLS(y, sm.add_constant(X_final))
res_final = mod_final.fit()
print(res_final.summary())

                            OLS Regression Results                            
Dep. Variable:                chinRyo   R-squared:                       0.842
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     170.8
Date:                Mon, 06 Oct 2025   Prob (F-statistic):          5.95e-110
Time:                        17:25:10   Log-Likelihood:                -3177.4
No. Observations:                 298   AIC:                             6375.
Df Residuals:                     288   BIC:                             6412.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.014e+04   3100.751      3.269      0.0

In [64]:
Sigma = np.asmatrix(X_final.cov())
def Mahala2(vec_x, vec_mean, mat):
    length = mat.shape[0]
    vec_x = np.array(vec_x, dtype='float64')
    vec = np.asmatrix((vec_x - vec_mean).values.reshape(length, 1))
    inv = np.linalg.inv(mat)
    mahala2 = vec.T.dot(inv.dot(vec))
    return mahala2[0, 0]

In [65]:
n = len(X_final)
dfm = res_final.df_model
t_0025 = ss.t.isf(q=0.05/2, df=n-dfm-1)
vec_mean = X_final.mean()
print(vec_mean)

senMenseki    29.050000
autolock       0.345638
parking        0.107383
aircon         0.919463
shinchiku      0.043624
kai_all        4.140940
kai_above      0.748322
bathtoilet     0.714765
addbatch       0.221477
dtype: float64


In [66]:
X = sm.add_constant(X_final).iloc[0,:] 
print(X)
hat_y=X.dot(res_final.params) 
print(hat_y)

D2_0 = Mahala2(X_final.iloc[0,:], vec_mean, Sigma)

Ve = res_final.scale 
Se_o = np.sqrt((1/n + D2_0 / (n-1)) * Ve)

ci_low = hat_y - t_0025 * Se_o# 
ci_up = hat_y + t_0025 * Se_o

print("理論値:", hat_y)
print("理論値の信頼区間：({0},{1})".format(ci_low,ci_up))

const          1.0
senMenseki    19.8
autolock       0.0
parking        0.0
aircon         1.0
shinchiku      0.0
kai_all        2.0
kai_above      1.0
bathtoilet     1.0
addbatch       1.0
Name: 0, dtype: float64
61825.39652345114
理論値: 61825.39652345114
理論値の信頼区間：(57554.70767732895,66096.08536957332)


In [67]:
X_test2 = X_test[['senMenseki', 'autolock', 'parking', 'aircon', 'shinchiku', 'kai_all', 'kai_above', 'bathtoilet', 'addbatch']]
X = sm.add_constant(X_test2, has_constant='add').iloc[0,:]
hat_y=X.dot(res_final.params)

D2_0 = Mahala2(X_test2.iloc[0,:], vec_mean, Sigma) 
Se_p = np.sqrt((1 + 1/n + D2_0 / (n-1)) * Ve)

pi_low = hat_y - t_0025 * Se_p
pi_up = hat_y + t_0025 * Se_p

print("予測値:", hat_y)
print("予測値の信頼区間：({0},{1})".format(pi_low,pi_up))

予測値: 113059.08021879205
予測値の信頼区間：(90996.3839847583,135121.7764528258)
